In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import torch
from monai.transforms import *

In [ ]:
file_path = r"E:\Data\ADNI\adni-fnirt-corrected\brain_adni_0021_I338266_fsld.nii.gz"

In [ ]:
# 使用nibabel加载nii.gz文件
nii_img = nib.load(file_path)
nii_img_process = nii_img.get_fdata()
nii_img_process = nii_img_process.astype(np.float32)
nii_img_process = torch.from_numpy(nii_img_process)
nii_img_process = nii_img_process.unsqueeze(0)
print(nii_img_process.shape)

size = 100
data_transforms_resize = Compose([
    Resize(spatial_size=[size, size, size]),
])

prob = 1
data_transforms = Compose([
    # RandRotate90(prob=prob, spatial_axes=[1, 2]),
    # RandRotate90(prob=prob, spatial_axes=[0, 1]),
    # RandRotate90(prob=prob, spatial_axes=[0, 2]),
    # RandFlip(prob=prob, spatial_axis=0),
    # RandFlip(prob=prob, spatial_axis=1),
    # RandFlip(prob=prob, spatial_axis=2),
    
    RandAdjustContrast(prob=prob, gamma=(0.7, 1.3)),
    RandGaussianNoise(prob=prob),
    # RandAffine(prob=prob, translate_range=10, scale_range=(0.9, 1.1), rotate_range=45),
    
    Resize(spatial_size=[size, size, size]),
    NormalizeIntensity(channel_wise=True),
])

nii_img_resize = data_transforms_resize(nii_img_process).squeeze(0).numpy()
nii_img_process = data_transforms(nii_img_process).squeeze(0).numpy()

print(nii_img_process.shape)

In [ ]:
print(nii_img_process[40][40])

In [ ]:
# image为numpy数组
def visualize(image, is_save = False, is_information = False, nii_img=None):

    # 打印图像维度
    print("Image shape:", image.shape)

    # 可视化每一层切片
    num_slices = image.shape[-1]

    # 设置子图的行数和列数
    num_rows = num_slices // 10 + 1  # 每行显示10个切片
    num_cols = min(num_slices, 10)

    # 设置子图的大小
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))

    # 遍历每一层切片并可视化
    for i in range(num_slices):
        row_idx = i // 10
        col_idx = i % 10

        # 在子图中显示每一层切片
        axes[row_idx, col_idx].imshow(np.flipud(image[:,:,i].T), cmap='gray', vmin=0, vmax=1)
        axes[row_idx, col_idx].axis('off')  # 关闭坐标轴

    # 如果切片数量不是10的倍数，隐藏多余的子图
    for i in range(num_slices, num_rows * num_cols):
        row_idx = i // 10
        col_idx = i % 10
        fig.delaxes(axes[row_idx, col_idx])

    plt.show()

    # 保存切片
    if is_save:
        print(f"num of slices:{num_slices}")
        plt.clf()
        for i in range(num_slices):
            # plt.imshow(image[:, :, i])
            plt.imsave(f"./image/slice_{i+1}.jpg", image[:, :, i], cmap='gray', vmin=0, vmax=1)
            


    if is_information:
    # 查看图像大小
        height, width, depth = image.shape
        print(f"The image object height: {height}, width:{width}, depth:{depth}")
        # 查看图像值范围
        print(f'image value range: [{image.min()}, {image.max()}]')

        # 查看图像成像信息，如 层厚，平面（in-plane）分辨率等

        # 矩阵以外的信息可以通过 image_obj.header 获取

        # header是键值对，查看 header 包含的所有信息
        print('headers', nii_img.header.keys())
        # 查看成像信息
        pixdim = nii_img.header['pixdim']
        print(f'z轴分辨率： {pixdim[3]}')
        print(f'in plane 分辨率： {pixdim[1]} * {pixdim[2]}')
        x_range = pixdim[1] * height
        y_range = pixdim[2] * width
        z_range = pixdim[3] * depth
        print(f"The image object x_range: {x_range}, y_range:{y_range}, z_range:{z_range}")
        # 整个数据
        print('img1_obj', nii_img)

In [ ]:
def normalize_image_0_to_1(image):
    """
    将输入的三维图像归一化到0-1范围。
    :param image: 三维Numpy数组，代表图像数据。
    :return: 归一化后的图像数据。
    """
    # 计算图像的最小值和最大值
    min_val = np.min(image)
    max_val = np.max(image)
    
    # 应用最大-最小归一化
    normalized_image = (image - min_val) / (max_val - min_val)
    
    return normalized_image

In [ ]:
visualize(normalize_image_0_to_1(nii_img.get_fdata()))

In [ ]:
visualize(normalize_image_0_to_1(nii_img_resize))

In [ ]:
nii_img_process_normailized = normalize_image_0_to_1(nii_img_process)
visualize(nii_img_process_normailized)

In [ ]:
nii_img_process_normailized[0][1]